In [ ]:
from collections import OrderedDict as odict
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
import datetime as dt

import ee
# import geemap.plotlymap as geemap
# from geemap import geemap

import colorcet
import param
import panel
import datashader as ds
import intake
import dask.dataframe

import hvplot.pandas

import holoviews
from holoviews.element import tiles
from holoviews.operation.datashader import rasterize, shade, spread, datashade

import seaborn
import datashader.transfer_functions as tf

from bokeh.models import HoverTool

In [ ]:
# This will become default in pandas 3.0
pd.options.mode.copy_on_write = True
panel.extension()
holoviews.extension('bokeh', logo=False)

ee.Authenticate()
ee.Initialize(project='sentinel-treeclassification')

In [ ]:
class SentinelGetter:
    def mask_s2_clouds(self, image):
      # Quality assessment with resolution in meters
      qa = image.select('QA60')
      # Bits 10 and 11 are clouds and cirrus, respectively.
      cloud_bit_mask = 1 << 10
      cirrus_bit_mask = 1 << 11
      # Both flags should be set to zero, indicating clear conditions.
      mask = (
          qa.bitwiseAnd(cloud_bit_mask)
          .eq(0)
          .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
      )
      return image.updateMask(mask)

    def get_image(self, center_date, bbox):
        modified_data = center_date.replace(month=9, year=center_date.year-1)
        month = pd.DateOffset(months=2)
        image = (
            ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterDate(modified_data - month, modified_data + month)
            # Pre-filter to get less cloudy granules.
            .map(self.mask_s2_clouds)
            .mean()
            .clip(bbox)
        )
        return image

In [ ]:
class Explorer(param.Parameterized):
    cmap_list = ['glasbey', 'fire', 'bgy', 'bgyw', 'bmy', 'gray', 'kbc']
    cmaps = odict([(n, colorcet.palette[n]) for n in cmap_list])
    color_map = param.Selector(objects=cmaps)

    date = param.Date(default=dt.date(2024, 3, 1))

    earth_url = 'https://mt1.google.com/vt/lyrs=y&x={X}&y={Y}&z={Z}'
    earth_map = [('Google Satellite', holoviews.Tiles(earth_url, name='Google Satellite'))]
    sentinel_map = [('Sentinel 2', holoviews.Tiles(name='Sentinel 2'))]
    other_maps = ['CartoLight', 'EsriImagery', 'OSM', 'CartoDark']
    bases = odict([(name, getattr(tiles, name)().relabel(name)) for name in other_maps] + earth_map + sentinel_map)
    basemap = param.Selector(objects=bases)
    
    data_opacity = param.Magnitude(0.75)  
    map_opacity = param.Magnitude(1.00)

    range_xy = holoviews.streams.RangeXY()
    
    gopts = holoviews.opts.Tiles(responsive=True, xaxis='bottom', yaxis='left', show_grid=True)

    catalog = intake.open_catalog(Path('catalog.yml'))
    datasets = list(catalog.keys())
    dataset = param.Selector(objects=datasets)

    @param.depends('dataset')
    def get_polygons(self):
        source = getattr(self.catalog, self.dataset)
        gdf = source.read()[source.metadata['usecols']]
        # WEB MERCATOR (EPSG: 3857) to match basemaps
        gdf = gdf.to_crs("epsg:3857")
        target = source.metadata['categories']['multi']
        gdf = gdf.rename(columns={target: 'target'})
        return holoviews.Polygons(gdf, vdims=['target'])
        
    @param.depends('map_opacity', 'basemap')
    def set_tiles(self):
        if 'Sentinel 2' in self.basemap.name:
            range_x = self.range_xy.contents['x_range']
            range_y = self.range_xy.contents['y_range']
            geojson_object = {
                'type': 'Polygon',
                'coordinates': [[[range_x[0], range_y[0]],[range_x[1], range_y[0]],
                                 [range_x[1], range_y[1]],[range_x[0], range_y[1]]]]
            }
            bbox = ee.Geometry(geojson_object, proj='epsg:3857')
            sentinel = SentinelGetter().get_image(self.date, bbox)
            rgb_bands = ['B4', 'B3', 'B2']
            # sentinel = sentinel.reproject("epsg:3857")
            
            map_id = sentinel.getMapId({'min':0, 'max': 3500, 'bands': rgb_bands})
            
            raw_url = ee.data.getTileUrl(map_id, int(0), int(50), 13)
            url = "/".join(raw_url.split('/')[:-3]) + '/{Z}/{X}/{Y}'
            print(url)
            return holoviews.Tiles(url, name='Sentinel 2').opts(self.gopts).opts(alpha=self.map_opacity)
        else:
            return self.basemap.opts(self.gopts).opts(alpha=self.map_opacity)
    
    def viewable(self, **kwargs): 
        #hover_tool = HoverTool(tooltips=[('Species', '@target')])
        polygons = holoviews.DynamicMap(self.get_polygons).apply.opts(
            # framewise=True, show_legend=True, legend_position='bottom',
            cmap=self.param.color_map,
            alpha=self.param.data_opacity, tools=['fullscreen'])

        rasterized = rasterize(polygons, aggregator='count_cat')
        shaded = shade(rasterized, color_key=self.param.color_map)
        dataplot = shaded.apply.opts(alpha=self.param.data_opacity,
                                     tools=['fullscreen'])

        dynamic_tiles = holoviews.DynamicMap(self.set_tiles)
        self.range_xy.source = dynamic_tiles
        
        return dynamic_tiles * dataplot

In [ ]:
explorer = Explorer(name="")

params = panel.Column(panel.Param(explorer.param, widgets={"date": panel.widgets.DatePicker}, expand_button=False))
dash = panel.Row(params, explorer.viewable())
dash.servable("Datashader Dashboard")

In [ ]:
# catalog = intake.open_catalog(Path('catalog.yml'))
# source = getattr(catalog, 'sample_treesat')
# gdf = source.read()[source.metadata['usecols']]
# gdf = gdf.to_crs("epsg:3857")
# #https://hvplot.holoviz.org/user_guide/Customization.html
# gdf.sample(10000).hvplot(
#     aggregator=ds.count_cat(source.metadata['categories']['general']),
#     # groupby=source.metadata['categories']['general'], 
#     datashade=True, rasterize=False, height=600, tiles='OSM', cmap='hot'
# )
# gdf.geometry = gdf.to_crs("epsg:3857").buffer(5, cap_style=3)

In [ ]:
# gdf.to_file("buffered_p.GeoJSON")

In [ ]:
# dash = panel.Row(hv)
# dash.servable("Datashader Dashboard")

In [ ]:
# holoviews.help(holoviews.Tiles)